# **Datasets Cleansing**

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

C:\Users\guanq\AppData\Local\Temp\ipykernel_35952\3172591061.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


**Our group used six datasets from '*DATA.NY.GOV*'(https://data.ny.gov/).**

**Congestion Zone:** https://data.ny.gov/Transportation/MTA-Congestion-Relief-Zone-Vehicle-Entries-Beginni/t6yz-b64h/about_data .

**Daily ridership:** https://data.ny.gov/Transportation/MTA-Daily-Ridership-and-Traffic-Beginning-2020/sayj-mze2/about_data .

**Tunnel and bridge crossings:** https://data.ny.gov/Transportation/MTA-Bridges-and-Tunnels-Hourly-Crossings-Beginning/ebfx-2m7v/about_data .

**Fare Evasion:** https://data.ny.gov/Transportation/MTA-NYCT-Subway-Fare-Evasion-Beginning-2018/6kj3-ijvb/about_data .

**Station entry/exit:** https://data.ny.gov/Transportation/MTA-Subway-Entrances-and-Exits-2024/i9wp-a4ja/about_data .

**Traffic violations:** https://data.ny.gov/Transportation/Traffic-Tickets-Issued-Four-Year-Window/q4hy-kbtf/about_data .


## **1. Congestion Zone cleansing**

In [2]:
df = pd.read_csv("congestion zone.csv")

# Check for missing values
missing_count = df.isnull().sum()
total_missing = missing_count.sum()
print(f"\nFound {total_missing} missing values")
if total_missing > 0:
    print("Missing values by column:")
    print(missing_count[missing_count > 0])

# Handle missing values
if total_missing > 0:
    # For numeric columns, fill with median
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numeric_cols:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].median())
            print(f"Filled missing values in {col} with median")

    # For categorical columns, fill with mode
    cat_cols = df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        if df[col].isnull().sum() > 0:
            mode_val = df[col].mode()[0]
            df[col] = df[col].fillna(mode_val)
            print(f"Filled missing values in {col} with mode: '{mode_val}'")

    # Verify missing values have been handled
    remaining_missing = df.isnull().sum().sum()
    print(f"Remaining missing values: {remaining_missing}")


Found 0 missing values


In [3]:
# Check for and remove duplicates
dup_count = df.duplicated().sum()
print(f"\nFound {dup_count} duplicate rows")

if dup_count > 0:
    df = df.drop_duplicates()
    print(f"Removed {dup_count} duplicate rows")


Found 0 duplicate rows


In [4]:
# Convert necessary data types
# Convert date columns to datetime
date_cols = [col for col in df.columns if 'Date' in col or 'date' in col]
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"Converted {col} to datetime")

# Convert numeric columns
numeric_cols = ['Minute of Hour', 'Hour of Day', 'CRZ Entries']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        print(f"Converted {col} to numeric")

Converted Toll Date to datetime
Converted Minute of Hour to numeric
Converted Hour of Day to numeric
Converted CRZ Entries to numeric


In [5]:
# Detect outliers in 'CRZ Entries'
if 'CRZ Entries' in df.columns:
    # Calculate quartiles
    Q1 = df['CRZ Entries'].quantile(0.25)
    Q3 = df['CRZ Entries'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define outlier boundaries
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers = (df['CRZ Entries'] < lower_bound) | (df['CRZ Entries'] > upper_bound)
    outlier_count = outliers.sum()
    print(f"\nIdentified {outlier_count} outliers in CRZ Entries")
    print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")
    
    # Summary before removing outliers
    print("\nSummary statistics before removing outliers:")
    print(df['CRZ Entries'].describe())
    
    # Remove outliers
    df_clean = df[~outliers]
    
    # Summary after removing outliers
    print("\nSummary statistics after removing outliers:")
    print(df_clean['CRZ Entries'].describe())
    
    print(f"\nRemoved {outlier_count} outliers. Final data shape: {df_clean.shape}")
else:
    print("'CRZ Entries' column not found")
    df_clean = df


Identified 58540 outliers in CRZ Entries
Lower bound: -87.0, Upper bound: 145.0

Summary statistics before removing outliers:
count    508032.000000
mean         45.857869
std          80.247676
min           0.000000
25%           0.000000
50%           4.000000
75%          58.000000
max         599.000000
Name: CRZ Entries, dtype: float64

Summary statistics after removing outliers:
count    449492.000000
mean         21.378494
std          36.253207
min           0.000000
25%           0.000000
50%           3.000000
75%          24.000000
max         145.000000
Name: CRZ Entries, dtype: float64

Removed 58540 outliers. Final data shape: (449492, 14)


In [6]:
# Final data quality check
print("\nFinal Data Quality Report:")
print(f"Total rows: {len(df_clean)}")
print(f"Missing values: {df_clean.isnull().sum().sum()}")
print(f"Duplicate row count: {df_clean.duplicated().sum()}")

df_clean.to_csv("cleaned_congestion_zone.csv", index=False)
print("\nSaved cleaned data to 'cleaned_congestion_zone.csv'")


Final Data Quality Report:
Total rows: 449492
Missing values: 0
Duplicate row count: 0

Saved cleaned data to 'cleaned_congestion_zone.csv'


## **2. Daily Ridership Cleansing**

In [7]:
df1 = pd.read_csv("Daily ridership.csv")

# Check for missing values
missing_count1 = df1.isnull().sum()
total_missing1 = missing_count1.sum()
print(f"\nFound {total_missing1} missing values")
if total_missing1 > 0:
    print("Missing values by column:")
    print(missing_count1[missing_count1 > 0])

# Handle missing values
if total_missing1 > 0:
    # For numeric columns, fill with median
    numeric_cols1 = df1.select_dtypes(include=['int64', 'float64']).columns
    for col1 in numeric_cols1:
        if df1[col1].isnull().sum() > 0:
            df1[col1] = df1[col1].fillna(df1[col1].median())
            print(f"Filled missing values in {col1} with median")
    
    # For categorical columns, fill with mode
    cat_cols1 = df1.select_dtypes(include=['object']).columns
    for col1 in cat_cols1:
        if df1[col1].isnull().sum() > 0:
            mode_val1 = df1[col1].mode()[0]
            df1[col1] = df1[col1].fillna(mode_val1)
            print(f"Filled missing values in {col1} with mode: '{mode_val1}'")
    
    # Verify missing values have been handled
    remaining_missing1 = df1.isnull().sum().sum()
    print(f"Remaining missing values: {remaining_missing1}")


Found 32 missing values
Missing values by column:
Count    32
dtype: int64
Filled missing values in Count with median
Remaining missing values: 0


In [8]:
# Check for and remove duplicates
dup_count1 = df1.duplicated().sum()
print(f"\nFound {dup_count1} duplicate rows")

if dup_count1 > 0:
    df1 = df1.drop_duplicates()
    print(f"Removed {dup_count1} duplicate rows")


Found 0 duplicate rows


In [9]:
# Convert necessary data types
# Convert Date to datetime
if 'Date' in df1.columns:
    df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')
    print("Converted Date to datetime")

# Ensure Count is numeric
if 'Count' in df1.columns:
    df1['Count'] = pd.to_numeric(df1['Count'], errors='coerce')
    print("Converted Count to numeric")

Converted Date to datetime
Converted Count to numeric


In [10]:
# Detect outliers in 'Count'
if 'Count' in df1.columns:
    modes1 = df1['Mode'].unique()
    outliers1 = pd.Series(False, index=df1.index)
    
    print("\nOutlier detection by mode:")
    for mode1 in modes1:
        mode_data1 = df1[df1['Mode'] == mode1]['Count']
        if len(mode_data1) >= 4:  
            Q1_1 = mode_data1.quantile(0.25)
            Q3_1 = mode_data1.quantile(0.75)
            IQR1 = Q3_1 - Q1_1
            
            lower_bound1 = Q1_1 - 1.5 * IQR1
            upper_bound1 = Q3_1 + 1.5 * IQR1
            
            mode_outliers1 = (mode_data1 < lower_bound1) | (mode_data1 > upper_bound1)
            outlier_count1 = mode_outliers1.sum()
            
            print(f"Mode {mode1}: {outlier_count1} outliers")
            print(f"  Bounds: {lower_bound1} - {upper_bound1}")
            
            outliers1 = outliers1 | (df1['Mode'] == mode1) & mode_outliers1
    
    total_outliers1 = outliers1.sum()
    print(f"\nTotal outliers across all modes: {total_outliers1}")
    
    # Summary before removing outliers
    print("\nSummary statistics before removing outliers:")
    print(df1['Count'].describe())
    
    # Remove outliers
    df_clean1 = df1[~outliers1]
    
    # Summary after removing outliers
    print("\nSummary statistics after removing outliers:")
    print(df_clean1['Count'].describe())
    
    print(f"\nRemoved {total_outliers1} outliers. Final data shape: {df_clean1.shape}")
else:
    print("'Count' column not found")
    df_clean1 = df1


Outlier detection by mode:
Mode AAR: 2 outliers
  Bounds: -1222.0 - 46322.0
Mode SIR: 41 outliers
  Bounds: -4770.0 - 13918.0
Mode MNR: 0 outliers
  Bounds: -133665.0 - 372663.0
Mode Subway: 0 outliers
  Bounds: -887492.0 - 6128636.0
Mode LIRR: 0 outliers
  Bounds: -103329.0 - 388815.0
Mode Bus: 0 outliers
  Bounds: -218282.0 - 2296790.0
Mode BT: 117 outliers
  Bounds: 612878.0 - 1146910.0
Mode CBD Entries: 3 outliers
  Bounds: 462565.5 - 624145.5
Mode CRZ Entries: 3 outliers
  Bounds: 406000.625 - 554713.625

Total outliers across all modes: 166

Summary statistics before removing outliers:
count    1.285900e+04
mean     6.738839e+05
std      9.633814e+05
min      2.000000e+00
25%      2.417750e+04
50%      1.853480e+05
75%      9.500055e+05
max      5.498809e+06
Name: Count, dtype: float64

Summary statistics after removing outliers:
count    1.269300e+04
mean     6.778974e+05
std      9.688226e+05
min      2.000000e+00
25%      2.397200e+04
50%      1.823140e+05
75%      9.539330e+

In [11]:
# Final data quality check
print("\nFinal Data Quality Report:")
print(f"Total rows: {len(df_clean1)}")
print(f"Missing values: {df_clean1.isnull().sum().sum()}")
print(f"Duplicate row count: {df_clean1.duplicated().sum()}")

# Distribution by Mode
if 'Mode' in df_clean1.columns:
    print("\nDistribution by Mode:")
    print(df_clean1['Mode'].value_counts())

df_clean1.to_csv("cleaned_daily_ridership.csv", index=False)
print("\nSaved cleaned data to 'cleaned_daily_ridership.csv'")


Final Data Quality Report:
Total rows: 12693
Missing values: 0
Duplicate row count: 0

Distribution by Mode:
Mode
MNR            1825
Subway         1825
LIRR           1825
Bus            1825
AAR            1823
SIR            1784
BT             1708
CBD Entries      39
CRZ Entries      39
Name: count, dtype: int64

Saved cleaned data to 'cleaned_daily_ridership.csv'


## **3. Tunning and Bridge crossings cleansing**

In [12]:
df2 = pd.read_csv("Tunnel and Bridge Crossings.csv")


# Check for missing values
missing_count2 = df2.isnull().sum()
total_missing2 = missing_count2.sum()
print(f"\nFound {total_missing2} missing values")
if total_missing2 > 0:
    print("Missing values by column:")
    print(missing_count2[missing_count2 > 0])

# Handle missing values
if total_missing2 > 0:
    # For numeric columns, fill with median
    numeric_cols2 = df2.select_dtypes(include=['int64', 'float64']).columns
    for col2 in numeric_cols2:
        if df2[col2].isnull().sum() > 0:
            df2[col2] = df2[col2].fillna(df2[col2].median())
            print(f"Filled missing values in {col2} with median")
    
    # For categorical columns, fill with mode
    cat_cols2 = df2.select_dtypes(include=['object']).columns
    for col2 in cat_cols2:
        if df2[col2].isnull().sum() > 0:
            mode_val2 = df2[col2].mode()[0]
            df2[col2] = df2[col2].fillna(mode_val2)
            print(f"Filled missing values in {col2} with mode: '{mode_val2}'")
    
    # Verify missing values have been handled
    remaining_missing2 = df2.isnull().sum().sum()
    print(f"Remaining missing values: {remaining_missing2}")


Found 0 missing values


In [13]:
# Check for and remove duplicates
dup_count2 = df2.duplicated().sum()
print(f"\nFound {dup_count2} duplicate rows")

if dup_count2 > 0:
    df2 = df2.drop_duplicates()
    print(f"Removed {dup_count2} duplicate rows")


Found 0 duplicate rows


In [14]:
# Convert necessary data types
# Convert date columns to datetime
date_cols2 = [col2 for col2 in df2.columns if 'Date' in col2 or 'date' in col2]
for col2 in date_cols2:
    df2[col2] = pd.to_datetime(df2[col2], errors='coerce')
    print(f"Converted {col2} to datetime")

# Convert Traffic Count to numeric
if 'Traffic Count' in df2.columns:
    df2['Traffic Count'] = pd.to_numeric(df2['Traffic Count'], errors='coerce')
    print("Converted Traffic Count to numeric")

Converted Date to datetime
Converted Traffic Count to numeric


In [15]:
# Detect outliers in 'Traffic Count'
if 'Traffic Count' in df2.columns:
    # Calculate quartiles
    Q1_2 = df2['Traffic Count'].quantile(0.25)
    Q3_2 = df2['Traffic Count'].quantile(0.75)
    IQR2 = Q3_2 - Q1_2
    
    # Define outlier boundaries
    lower_bound2 = Q1_2 - 1.5 * IQR2
    upper_bound2 = Q3_2 + 1.5 * IQR2
    
    # Identify outliers
    outliers2 = (df2['Traffic Count'] < lower_bound2) | (df2['Traffic Count'] > upper_bound2)
    outlier_count2 = outliers2.sum()
    print(f"\nIdentified {outlier_count2} outliers in Traffic Count")
    print(f"Lower bound: {lower_bound2}, Upper bound: {upper_bound2}")
  
    print("\nSummary statistics before removing outliers:")
    print(df2['Traffic Count'].describe())
    
    # Remove outliers
    df_clean2 = df2[~outliers2]
    
    # Summary after removing outliers
    print("\nSummary statistics after removing outliers:")
    print(df_clean2['Traffic Count'].describe())
    
    print(f"\nRemoved {outlier_count2} outliers. Final data shape: {df_clean2.shape}")
else:
    print("'Traffic Count' column not found")
    df_clean2 = df2


Identified 1941497 outliers in Traffic Count
Lower bound: -38.5, Upper bound: 69.5

Summary statistics before removing outliers:
count    1.076610e+07
mean     1.730310e+02
std      6.067171e+02
min      1.000000e+00
25%      2.000000e+00
50%      6.000000e+00
75%      2.900000e+01
max      1.452200e+04
Name: Traffic Count, dtype: float64

Summary statistics after removing outliers:
count    8.824599e+06
mean     9.229330e+00
std      1.282092e+01
min      1.000000e+00
25%      1.000000e+00
50%      4.000000e+00
75%      1.100000e+01
max      6.900000e+01
Name: Traffic Count, dtype: float64

Removed 1941497 outliers. Final data shape: (8824599, 11)


In [16]:
# Final data quality check
print("\nFinal Data Quality Report:")
print(f"Total rows: {len(df_clean2)}")
print(f"Missing values: {df_clean2.isnull().sum().sum()}")
print(f"Duplicate row count: {df_clean2.duplicated().sum()}")

df_clean2.to_csv("cleaned_tunnel_bridge_crossings.csv", index=False)
print("\nSaved cleaned data to 'cleaned_tunnel_bridge_crossings.csv'")


Final Data Quality Report:
Total rows: 8824599
Missing values: 0
Duplicate row count: 0

Saved cleaned data to 'cleaned_tunnel_bridge_crossings.csv'


## **4. Fare Evasion cleansing**

In [17]:
import pandas as pd

df3 = pd.read_csv("Fare evasion.csv")

# Standardize column names
df3.columns = df3.columns.str.strip().str.replace(' ', '_')

# Process Time Period column
if 'Time_Period' in df3.columns:
    # Extract year and quarter
    df3['Year3'] = df3['Time_Period'].str.extract(r'(\d{4})').astype(int)
    df3['Quarter3'] = df3['Time_Period'].str.extract(r'Q(\d)').astype(int)
    
    # Calculate a decimal date for time series analysis 
    df3['Decimal_Year3'] = df3['Year3'] + (df3['Quarter3'] - 1) * 0.25
    
    # Create a proper datetime for the start of each quarter
    df3['Date3'] = pd.to_datetime(df3['Year3'].astype(str) + '-' + 
                                (df3['Quarter3'] * 3 - 2).astype(str) + '-01')

In [18]:
# Handle missing values
missing_before3 = df3.isnull().sum().sum()
if 'Fare_Evasion' in df3.columns and df3['Fare_Evasion'].isnull().any():
    df3['Fare_Evasion'] = df3['Fare_Evasion'].interpolate(method='linear')
    print("Interpolated missing Fare_Evasion values")

if 'Margin_of_Error' in df3.columns and df3['Margin_of_Error'].isnull().any():
    available_margins3 = df3['Margin_of_Error'].dropna()
    if len(available_margins3) > 0:
        avg_margin3 = available_margins3.mean()
        df3['Margin_of_Error'] = df3['Margin_of_Error'].fillna(avg_margin3)
        print(f"Filled missing Margin_of_Error values with average: {avg_margin3:.4f}")

missing_after3 = df3.isnull().sum().sum()
print(f"Filled {missing_before3 - missing_after3} missing values")

Interpolated missing Fare_Evasion values
Filled missing Margin_of_Error values with average: 0.0112
Filled 10 missing values


In [19]:
# Detect and remove outliers
if 'Fare_Evasion' in df3.columns:
    # Calculate quartiles and IQR
    Q1_3 = df3['Fare_Evasion'].quantile(0.25)
    Q3_3 = df3['Fare_Evasion'].quantile(0.75)
    IQR3 = Q3_3 - Q1_3
    
    # Define outlier boundaries
    lower_bound3 = Q1_3 - 1.5 * IQR3
    upper_bound3 = Q3_3 + 1.5 * IQR3
    
    # Identify outliers
    outliers3 = (df3['Fare_Evasion'] < lower_bound3) | (df3['Fare_Evasion'] > upper_bound3)
    outlier_count3 = outliers3.sum()
    
    # Save the time periods with outliers for reference
    if outlier_count3 > 0:
        outlier_periods3 = df3.loc[outliers3, 'Time_Period'].tolist()
        print(f"Identified {outlier_count3} outliers in the following time periods: {outlier_periods3}")
    
    # Remove outliers
    df3_original = df3.copy() 
    df3 = df3[~outliers3]
    print(f"Removed {outlier_count3} outliers, data shape after removal: {df3.shape}")

Removed 0 outliers, data shape after removal: (28, 7)


In [20]:
# Add derived features for analysis
if all(col in df3.columns for col in ['Fare_Evasion', 'Margin_of_Error']):
    df3['Upper_Bound3'] = df3['Fare_Evasion'] + df3['Margin_of_Error']
    df3['Lower_Bound3'] = df3['Fare_Evasion'] - df3['Margin_of_Error']
    df3['Lower_Bound3'] = df3['Lower_Bound3'].apply(lambda x: max(0, x))
    print("Added Upper_Bound3 and Lower_Bound3 columns")

# Add categorical period features for comparison
if 'Year3' in df3.columns:
    df3['COVID_Period3'] = 'Pre-COVID'
    df3.loc[(df3['Year3'] == 2020) & (df3['Quarter3'] >= 1) | 
            (df3['Year3'] == 2021) & (df3['Quarter3'] <= 2), 'COVID_Period3'] = 'During-COVID'
    df3.loc[(df3['Year3'] > 2021) | 
            (df3['Year3'] == 2021) & (df3['Quarter3'] > 2), 'COVID_Period3'] = 'Post-COVID'
    print("Added COVID_Period3 classification")

# Add year-over-year change
if 'Fare_Evasion' in df3.columns and 'Quarter3' in df3.columns:
    df3 = df3.sort_values(['Year3', 'Quarter3'])
    
    df3['YoY_Change3'] = df3.groupby('Quarter3')['Fare_Evasion'].pct_change(4) * 100
    print("Added YoY_Change3 column (percentage change from same quarter previous year)")

# Add quarter-over-quarter change
if 'Fare_Evasion' in df3.columns:
    df3['QoQ_Change3'] = df3['Fare_Evasion'].pct_change() * 100
    print("Added QoQ_Change3 column (percentage change from previous quarter)")

Added Upper_Bound3 and Lower_Bound3 columns
Added COVID_Period3 classification
Added YoY_Change3 column (percentage change from same quarter previous year)
Added QoQ_Change3 column (percentage change from previous quarter)


In [21]:
# Format percentage columns for better readability
percentage_cols3 = ['Fare_Evasion', 'Margin_of_Error', 'Upper_Bound3', 'Lower_Bound3']
for col3 in percentage_cols3:
    if col3 in df3.columns:
        df3[f'{col3}_Pct'] = df3[col3] * 100
        print(f"Added {col3}_Pct column with percentage format")

Added Fare_Evasion_Pct column with percentage format
Added Margin_of_Error_Pct column with percentage format
Added Upper_Bound3_Pct column with percentage format
Added Lower_Bound3_Pct column with percentage format


In [22]:
# Calculate summary statistics by year and COVID period
year_stats3 = df3.groupby('Year3')['Fare_Evasion'].agg(['mean', 'min', 'max']).reset_index()
year_stats3.columns = ['Year3', 'Average_Evasion3', 'Min_Evasion3', 'Max_Evasion3']

covid_stats3 = df3.groupby('COVID_Period3')['Fare_Evasion'].agg(['mean', 'min', 'max']).reset_index()
covid_stats3.columns = ['COVID_Period3', 'Average_Evasion3', 'Min_Evasion3', 'Max_Evasion3']

print("\nYearly statistics generated")
print("\nCOVID period statistics generated")

# Display the final data structure
print(f"\nFinal cleaned data shape: {df3.shape}")
print("\nColumn Data Types:")
print(df3.dtypes)
print("\nSample of cleaned data:")
print(df3.head())

# Compare statistics before and after outlier removal
if 'df3_original' in locals():
    print("\nStatistics comparison before and after outlier removal:")
    print("Before removal:", df3_original['Fare_Evasion'].describe().round(4))
    print("After removal:", df3['Fare_Evasion'].describe().round(4))

print("\nData Quality Report:")
print(f"Total rows: {len(df3)}")
print(f"Missing values: {df3.isnull().sum().sum()}")

print("\nFare Evasion Summary Statistics:")
print(df3['Fare_Evasion'].describe())

print("\nYear-over-Year Summary:")
if 'YoY_Change3' in df3.columns:
    print(df3.groupby('Year3')['YoY_Change3'].mean().round(2))


Yearly statistics generated

COVID period statistics generated

Final cleaned data shape: (28, 16)

Column Data Types:
Time_Period                    object
Fare_Evasion                  float64
Margin_of_Error               float64
Year3                           int32
Quarter3                        int32
Decimal_Year3                 float64
Date3                  datetime64[ns]
Upper_Bound3                  float64
Lower_Bound3                  float64
COVID_Period3                  object
YoY_Change3                   float64
QoQ_Change3                   float64
Fare_Evasion_Pct              float64
Margin_of_Error_Pct           float64
Upper_Bound3_Pct              float64
Lower_Bound3_Pct              float64
dtype: object

Sample of cleaned data:
  Time_Period  Fare_Evasion  Margin_of_Error  Year3  Quarter3  Decimal_Year3  \
0     2018-Q1         0.028         0.011158   2018         1        2018.00   
1     2018-Q2         0.029         0.011158   2018         2        2018

In [23]:
# Save the cleaned data
df3.to_csv("cleaned_fare_evasion.csv", index=False)
print("\nSaved cleaned data to 'cleaned_fare_evasion.csv'")

# Save statistics tables
year_stats3.to_csv("fare_evasion_yearly_stats.csv", index=False)
covid_stats3.to_csv("fare_evasion_covid_period_stats.csv", index=False)
print("Saved summary statistics tables")


Saved cleaned data to 'cleaned_fare_evasion.csv'
Saved summary statistics tables


## **5. Station entry/exit cleansing**

In [24]:
df4 = pd.read_csv("Station entry_exit.csv")

print("Original 'Exit Allowed' values:")
print(df4['Exit Allowed'].value_counts())
print("Name: count, dtype: int64")

# Standardize column names
df4.columns = df4.columns.str.strip().str.replace(' ', '_')

# Handle other missing values without touching Exit_Allowed
text_columns4 = df4.select_dtypes(include=['object']).columns
for col4 in text_columns4:
    if col4 != 'Exit_Allowed': 
        df4[col4] = df4[col4].fillna('Unknown')

# For numeric columns
numeric_columns4 = df4.select_dtypes(include=['float64', 'int64']).columns
for col4 in numeric_columns4:
    df4[col4] = df4[col4].fillna(df4[col4].median())

Original 'Exit Allowed' values:
Exit Allowed
YES    2113
NO        7
Name: count, dtype: int64
Name: count, dtype: int64


In [25]:
# Handle coordinate columns
coordinate_cols4 = [col4 for col4 in df4.columns if ('entrance_georeference' in col4.lower() or 
                                                 'point' in col4.lower())]

for col4 in coordinate_cols4:
    if col4 in df4.columns and df4[col4].dtype == 'object':
        if df4[col4].astype(str).str.contains('POINT').any():
            extracted4 = df4[col4].astype(str).str.extract(r'POINT \(([^)]+)\)')
            df4[[f'{col4}_lon', f'{col4}_lat']] = extracted4[0].str.split(' ', expand=True)
            
            df4[f'{col4}_lon'] = pd.to_numeric(df4[f'{col4}_lon'], errors='coerce')
            df4[f'{col4}_lat'] = pd.to_numeric(df4[f'{col4}_lat'], errors='coerce')
            print(f"Extracted longitude and latitude from {col4}")

Extracted longitude and latitude from entrance_georeference


In [26]:
# Remove duplicates
before_rows4 = len(df4)
df4 = df4.drop_duplicates()
after_rows4 = len(df4)
print(f"Removed {before_rows4 - after_rows4} duplicate rows")

Removed 0 duplicate rows


In [27]:
# Detect and remove outliers in coordinate columns
outlier_cols4 = [col4 for col4 in df4.columns if ('_lon' in col4 or '_lat' in col4) and df4[col4].dtype != 'object']

# Combined outlier flag
df4['has_outlier4'] = False

# Process each coordinate column
for col4 in outlier_cols4:
    if df4[col4].isna().all():
        continue
    
    Q1_4 = df4[col4].quantile(0.25)
    Q3_4 = df4[col4].quantile(0.75)
    IQR4 = Q3_4 - Q1_4
    
    lower_bound4 = Q1_4 - 1.5 * IQR4
    upper_bound4 = Q3_4 + 1.5 * IQR4
    
    outlier_flag4 = (df4[col4] < lower_bound4) | (df4[col4] > upper_bound4)
    df4[f'{col4}_outlier4'] = outlier_flag4
    df4['has_outlier4'] = df4['has_outlier4'] | outlier_flag4
    
    print(f"Identified {outlier_flag4.sum()} outliers in {col4}")

# Remove outliers
df4_no_outliers = df4[~df4['has_outlier4']]
print(f"Removed {len(df4) - len(df4_no_outliers)} outlier rows")

Identified 98 outliers in entrance_georeference_lon
Identified 58 outliers in entrance_georeference_lat
Removed 120 outlier rows


In [28]:
# Add features
if 'Entrance' in df4_no_outliers.columns and 'Exit_Allowed' in df4_no_outliers.columns:
    df4_no_outliers['Access_Type4'] = 'Unknown'
    df4_no_outliers.loc[(df4_no_outliers['Entrance'] == 'YES') & (df4_no_outliers['Exit_Allowed'] == 'YES'), 'Access_Type4'] = 'Both'
    df4_no_outliers.loc[(df4_no_outliers['Entrance'] == 'YES') & (df4_no_outliers['Exit_Allowed'] != 'YES'), 'Access_Type4'] = 'Entrance Only'
    df4_no_outliers.loc[(df4_no_outliers['Entrance'] != 'YES') & (df4_no_outliers['Exit_Allowed'] == 'YES'), 'Access_Type4'] = 'Exit Only'
    df4_no_outliers.loc[(df4_no_outliers['Entrance'] != 'YES') & (df4_no_outliers['Exit_Allowed'] != 'YES'), 'Access_Type4'] = 'No Public Access'
    
    print("Created Access_Type4 feature")
    print(df4_no_outliers['Access_Type4'].value_counts())

In [29]:
# Final remove outlier flags
outlier_flag_cols4 = [col4 for col4 in df4_no_outliers.columns if col4.endswith('_outlier4')]
df4_no_outliers = df4_no_outliers.drop(columns=outlier_flag_cols4 + ['has_outlier4'])

In [30]:
# Final data quality report
print("\nFinal data shape:", df4_no_outliers.shape)
print("\nSample of cleaned data:")
print(df4_no_outliers.head())

print("\nData Quality Report:")
print(f"Total rows: {len(df4_no_outliers)}")
print(f"Missing values: {df4_no_outliers.isnull().sum().sum()}")
print(f"Duplicate row count: {df4_no_outliers.duplicated().sum()}")

df4_no_outliers.to_csv("cleaned_station_entry_exit.csv", index=False)
print("\nSaved cleaned data to 'cleaned_station_entry_exit.csv'")


Final data shape: (2000, 17)

Sample of cleaned data:
  Division    Line Borough                 Stop_Name  Complex_ID  \
0      BMT  4th Av       B  Atlantic Av-Barclays Ctr         617   
1      BMT  4th Av       B  Atlantic Av-Barclays Ctr         617   
2      BMT  4th Av       B  Atlantic Av-Barclays Ctr         617   
3      BMT  4th Av       B                  Union St          28   
4      BMT  4th Av       B                  Union St          28   

   Constituent_Station_Name  Station_ID GTFS_Stop_ID     Daytime_Routes  \
0  Atlantic Av-Barclays Ctr          27          R31  2 3 4 5 B D N Q R   
1  Atlantic Av-Barclays Ctr          27          R31  2 3 4 5 B D N Q R   
2  Atlantic Av-Barclays Ctr          27          R31  2 3 4 5 B D N Q R   
3                  Union St          28          R32                  R   
4                  Union St          28          R32                  R   

  Entrance_Type Entry_Allowed Exit_Allowed  Entrance_Latitude  \
0         Stair     

Missing values: 0
Duplicate row count: 0

Saved cleaned data to 'cleaned_station_entry_exit.csv'


## **6. Traffic Violations**

In [31]:
df5 = pd.read_csv("Traffic violations.csv")

print("Original column names:", df5.columns.tolist())
print("\nOriginal data types:")
print(df5.dtypes)

# Standardize column names
df5.columns = df5.columns.str.strip().str.replace(' ', '_')
print("\nStandardized column names:", df5.columns.tolist())

Original column names: ['Violation Charged Code', 'Violation Description', 'Violation Year', 'Violation Month', 'Violation Day of Week', 'Age at Violation', 'Gender', 'State of License', 'Police Agency', 'Court', 'Source']

Original data types:
Violation Charged Code     object
Violation Description      object
Violation Year              int64
Violation Month             int64
Violation Day of Week      object
Age at Violation          float64
Gender                     object
State of License           object
Police Agency              object
Court                      object
Source                     object
dtype: object

Standardized column names: ['Violation_Charged_Code', 'Violation_Description', 'Violation_Year', 'Violation_Month', 'Violation_Day_of_Week', 'Age_at_Violation', 'Gender', 'State_of_License', 'Police_Agency', 'Court', 'Source']


In [32]:
# Check for missing values
missing_before5 = df5.isnull().sum()
total_missing_before5 = missing_before5.sum()
print(f"\nFound {total_missing_before5} missing values before cleaning")
print("Missing values by column (top 5):")
print(missing_before5.sort_values(ascending=False).head())


Found 163950 missing values before cleaning
Missing values by column (top 5):
Age_at_Violation          163950
Violation_Charged_Code         0
Violation_Description          0
Violation_Year                 0
Violation_Month                0
dtype: int64


In [33]:
# Check for and handle inconsistent values
if 'State_of_X' in df5.columns:
    print("\nUnique values in State_of_X:")
    print(df5['State_of_X'].value_counts())
    
    # Standardize state names
    state_mapping5 = {
        'NEW YORK': 'NY',
        'NEW JERSEY': 'NJ',
        'PENNSYLVANIA': 'PA',
        'UNKNOWN': 'UNKNOWN'
    }
    df5['State_Standardized5'] = df5['State_of_X'].map(state_mapping5)
    print("\nStandardized states:")
    print(df5['State_Standardized5'].value_counts())

In [34]:
# Handle missing values
# For categorical columns
cat_columns5 = df5.select_dtypes(include=['object']).columns
for col5 in cat_columns5:
    if 'state' in col5.lower():
        df5[col5] = df5[col5].fillna('UNKNOWN')
    else:
        most_frequent5 = df5[col5].mode()[0]
        df5[col5] = df5[col5].fillna(most_frequent5)
        print(f"Filled missing values in {col5} with '{most_frequent5}'")

# For numeric columns
num_columns5 = df5.select_dtypes(include=['int64', 'float64']).columns
for col5 in num_columns5:
    df5[col5] = df5[col5].fillna(df5[col5].median())
    print(f"Filled missing values in {col5} with median")

missing_after5 = df5.isnull().sum().sum()
print(f"\nFilled {total_missing_before5 - missing_after5} missing values")

Filled missing values in Violation_Charged_Code with '1180D'
Filled missing values in Violation_Description with 'SPEED IN ZONE'
Filled missing values in Violation_Day_of_Week with 'WEDNESDAY'
Filled missing values in Gender with 'M'
Filled missing values in Police_Agency with 'NYC POLICE DEPT'
Filled missing values in Court with 'NASSAU TRAFFIC/PARKING AGENCY'
Filled missing values in Source with 'TSLED'
Filled missing values in Violation_Year with median
Filled missing values in Violation_Month with median
Filled missing values in Age_at_Violation with median

Filled 163950 missing values


In [35]:
# Convert data types
date_columns5 = [col5 for col5 in df5.columns if 'date' in col5.lower() or 'year' in col5.lower()]
for col5 in date_columns5:
    if 'year' in col5.lower() and df5[col5].dtype != 'datetime64[ns]':
        df5[col5] = pd.to_numeric(df5[col5], errors='coerce')
        print(f"Converted {col5} to numeric")

if 'Age_at_Violation' in df5.columns:
    if df5['Age_at_Violation'].dtype == 'object':
        df5['Age_Numeric5'] = df5['Age_at_Violation'].str.extract('(\d+)').astype(float)
        df5['Gender_Extracted5'] = df5['Age_at_Violation'].str.extract('(\d+\s*(\w))').iloc[:, 1]
        print("Extracted numeric age and gender")
    else:
        df5['Age_Numeric5'] = df5['Age_at_Violation']

Converted Violation_Year to numeric


<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:11: SyntaxWarning: invalid escape sequence '\d'
C:\Users\guanq\AppData\Local\Temp\ipykernel_35952\3121041409.py:10: SyntaxWarning: invalid escape sequence '\d'
  df5['Age_Numeric5'] = df5['Age_at_Violation'].str.extract('(\d+)').astype(float)
C:\Users\guanq\AppData\Local\Temp\ipykernel_35952\3121041409.py:11: SyntaxWarning: invalid escape sequence '\d'
  df5['Gender_Extracted5'] = df5['Age_at_Violation'].str.extract('(\d+\s*(\w))').iloc[:, 1]


In [36]:
# Identify and remove duplicates
before_rows5 = len(df5)
df5 = df5.drop_duplicates()
after_rows5 = len(df5)
print(f"\nRemoved {before_rows5 - after_rows5} duplicate rows")


Removed 1157935 duplicate rows


In [37]:
# Detect outliers in numeric columns
if 'Age_Numeric5' in df5.columns:
    if not df5['Age_Numeric5'].isna().all():
        Q1_5 = df5['Age_Numeric5'].quantile(0.25)
        Q3_5 = df5['Age_Numeric5'].quantile(0.75)
        IQR5 = Q3_5 - Q1_5
        
        lower_bound5 = Q1_5 - 1.5 * IQR5
        upper_bound5 = Q3_5 + 1.5 * IQR5
        
        # Create outlier flag
        outlier_flag5 = (df5['Age_Numeric5'] < lower_bound5) | (df5['Age_Numeric5'] > upper_bound5)
        df5['Age_outlier5'] = outlier_flag5
        
        print(f"\nIdentified {outlier_flag5.sum()} outliers in Age_Numeric5")
        print(f"Age outlier bounds: {lower_bound5:.1f} - {upper_bound5:.1f}")
        print("Age outliers:")
        print(df5[outlier_flag5]['Age_Numeric5'].value_counts().head())

# Remove outliers
if 'Age_outlier5' in df5.columns:
    df5_original = df5.copy() 
    df5_no_outliers = df5[~df5['Age_outlier5']]
    print(f"\nRemoved {len(df5) - len(df5_no_outliers)} outlier rows")
    print(f"Data shape after removing outliers: {df5_no_outliers.shape}")
    
    # Compare age distribution before and after
    print("\nAge distribution before removing outliers:")
    print(df5['Age_Numeric5'].describe())
    print("\nAge distribution after removing outliers:")
    print(df5_no_outliers['Age_Numeric5'].describe())

    df5 = df5_no_outliers


Identified 59042 outliers in Age_Numeric5
Age outlier bounds: -5.0 - 75.0
Age outliers:
Age_Numeric5
76.0    10922
77.0     8763
78.0     7635
79.0     6614
80.0     5280
Name: count, dtype: int64

Removed 59042 outlier rows
Data shape after removing outliers: (8844876, 13)

Age distribution before removing outliers:
count    8.903918e+06
mean     3.649193e+01
std      1.393770e+01
min      1.600000e+01
25%      2.500000e+01
50%      3.300000e+01
75%      4.500000e+01
max      9.500000e+01
Name: Age_Numeric5, dtype: float64

Age distribution after removing outliers:
count    8.844876e+06
mean     3.620253e+01
std      1.352154e+01
min      1.600000e+01
25%      2.500000e+01
50%      3.300000e+01
75%      4.500000e+01
max      7.500000e+01
Name: Age_Numeric5, dtype: float64


In [38]:
# Add derived features for analysis
if 'Violation_Date' in df5.columns and pd.api.types.is_datetime64_dtype(df5['Violation_Date']):
    df5['Day_of_Week5'] = df5['Violation_Date'].dt.day_name()
    print("\nCreated Day_of_Week5 feature")
elif 'Violation_3' in df5.columns:  
    day_mapping5 = {
        1: 'Monday',
        2: 'Tuesday',
        3: 'Wednesday',
        4: 'Thursday',
        5: 'Friday',
        6: 'Saturday',
        7: 'Sunday'
    }
    df5['Day_of_Week5'] = df5['Violation_3'].map(day_mapping5)
    print("\nCreated Day_of_Week5 feature from Violation_3")

# Age groups for analysis
if 'Age_Numeric5' in df5.columns:
    bins5 = [0, 18, 25, 35, 45, 55, 65, 100]
    labels5 = ['Under 18', '18-24', '25-34', '35-44', '45-54', '55-64', '65+']
    df5['Age_Group5'] = pd.cut(df5['Age_Numeric5'], bins=bins5, labels=labels5)
    print("Created Age_Group5 feature")

Created Age_Group5 feature


In [39]:
# Final cleanup
columns_to_drop5 = ['Age_outlier5'] if 'Age_outlier5' in df5.columns else []
df5 = df5.drop(columns=columns_to_drop5)
print(f"\nRemoved {len(columns_to_drop5)} temporary columns")


Removed 1 temporary columns


In [40]:
# Final data quality report
print("\nFinal data shape:", df5.shape)
print("\nColumn Data Types:")
print(df5.dtypes)
print("\nSample of cleaned data:")
print(df5.head())

print("\nData Quality Report:")
print(f"Total rows: {len(df5)}")
print(f"Missing values: {df5.isnull().sum().sum()}")
print(f"Duplicate row count: {df5.duplicated().sum()}")

df5.to_csv("cleaned_traffic_violations.csv", index=False)
print("\nSaved cleaned data to 'cleaned_traffic_violations.csv'")


Final data shape: (8844876, 13)

Column Data Types:
Violation_Charged_Code      object
Violation_Description       object
Violation_Year               int64
Violation_Month              int64
Violation_Day_of_Week       object
Age_at_Violation           float64
Gender                      object
State_of_License            object
Police_Agency               object
Court                       object
Source                      object
Age_Numeric5               float64
Age_Group5                category
dtype: object

Sample of cleaned data:
  Violation_Charged_Code      Violation_Description  Violation_Year  \
0                  403A1       FLD YIELD PEDEST NYC            2020   
1                  1110A   DISOBEYED TRAFFIC DEVICE            2020   
2                1225C2A  OPERATING MV MOBILE PHONE            2020   
3                  4014B       COM VEH ON PKWAY NYC            2020   
4                1111D1N               NYC REDLIGHT            2020   

   Violation_Month Violati